In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0-dev20201215


# Get data

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# Create Model

In [3]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
        ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Save checkpoints during training

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)



# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

# Load trained weights

In [4]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3120 - sparse_categorical_accuracy: 0.0990
Untrained model, accuracy:  9.90%


In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4128 - sparse_categorical_accuracy: 0.8670
Restored model, accuracy: 86.70%


# Checkcallback Options

In [8]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [9]:
ls {checkpoint_dir}


 Volume in drive C is Windows
 Volume Serial Number is B21E-32E9

 Directory of C:\Users\77bis\Desktop\CS598-FinalProject\Code\Leo\Jupyter\TF_tutorial\training_2

12/28/2020  10:28 AM    <DIR>          .
12/28/2020  10:28 AM    <DIR>          ..
12/28/2020  10:28 AM                81 checkpoint
12/28/2020  10:28 AM         1,628,726 cp-0000.ckpt.data-00000-of-00001
12/28/2020  10:28 AM               402 cp-0000.ckpt.index
12/28/2020  10:28 AM         4,886,685 cp-0005.ckpt.data-00000-of-00001
12/28/2020  10:28 AM             1,222 cp-0005.ckpt.index
12/28/2020  10:28 AM         4,886,685 cp-0010.ckpt.data-00000-of-00001
12/28/2020  10:28 AM             1,222 cp-0010.ckpt.index
12/28/2020  10:28 AM         4,886,685 cp-0015.ckpt.data-00000-of-00001
12/28/2020  10:28 AM             1,222 cp-0015.ckpt.index
12/28/2020  10:28 AM         4,886,685 cp-0020.ckpt.data-00000-of-00001
12/28/2020  10:28 AM             1,222 cp-0020.ckpt.index
12/28/2020  10:28 AM         4,886,685 cp-0025.ckpt.da

In [10]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

# Load trained Weights 2

In [11]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4713 - sparse_categorical_accuracy: 0.8800
Restored model, accuracy: 88.00%


# Manually save weights

In [12]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4713 - sparse_categorical_accuracy: 0.8800
Restored model, accuracy: 88.00%


# Save the entire model

In [13]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6311 - sparse_categorical_accuracy: 0.4356
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4389 - sparse_categorical_accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.3263 - sparse_categorical_accuracy: 0.9224
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2204 - sparse_categorical_accuracy: 0.9484
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1335 - sparse_categorical_accuracy: 0.9780
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [28]:
ls {'saved_model\my_model'}

 Volume in drive C is Windows
 Volume Serial Number is B21E-32E9

 Directory of C:\Users\77bis\Desktop\CS598-FinalProject\Code\Leo\Jupyter\TF_tutorial\saved_model\my_model

12/28/2020  10:34 AM    <DIR>          .
12/28/2020  10:34 AM    <DIR>          ..
12/28/2020  10:34 AM    <DIR>          assets
12/28/2020  10:34 AM             6,579 keras_metadata.pb
12/28/2020  10:34 AM            84,607 saved_model.pb
12/28/2020  10:34 AM    <DIR>          variables
               2 File(s)         91,186 bytes
               4 Dir(s)  197,905,596,416 bytes free


In [29]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4378 - sparse_categorical_accuracy: 0.8670
Restored model, accuracy: 86.70%
(1000, 10)


In [31]:
# HDF5 Format

In [32]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6416 - sparse_categorical_accuracy: 0.4831
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4732 - sparse_categorical_accuracy: 0.8600
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2791 - sparse_categorical_accuracy: 0.9284
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2301 - sparse_categorical_accuracy: 0.9466
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1641 - sparse_categorical_accuracy: 0.9587


In [33]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [34]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4278 - sparse_categorical_accuracy: 0.8560
Restored model, accuracy: 85.60%
